# f-scLVM

In this notebook we illustrate how f-scLVM can be used to identify biological processes driving variability between cells. 

First, we load some modules and set some directories, we here use the same data as before. 

In [ ]:
import sys
sys.path.append('./')
import os
import scipy as SP
import cPickle as pickle
import fscLVM.core as fscLVM
import fscLVM.utils.utils as utils
import h5py
#from utils import *
%pylab inline

data_dir = './data/'
out_base = './../results/'

f-scLVM expects a hdf file containing the normalised, log transformed gene expression data as well as a set of annotations. We have put this information together in hdf files. Start off by running the notebook for the T-cell data analysed in the scLVM notebook. Next, use different annotations (REACTOME) instead of MSigDB). Finally, have a look at a data-set of mESC cells staged by cell cycle.

In [ ]:
dFile = 'Tcell_sfERCC.hdf5'
#name of annotation
anno = 'MSigDB'

#specify noise model
noise = 'gauss'

#number of hidden (unannotated variables)
nHidden = 1
#indices of known covariates
idx_known = []

idxCol=[0,1]

Next we need to load the relevant data from the hdf5 file. minGenes is the minimum number of genes in a pathway.

In [ ]:
data = utils.load_data(dFile, annotation=anno, minGenes=15, nHidden=nHidden, doFast=True, noise=noise, data_dir=data_dir)

Next, we initialise the model and iterate.


In [ ]:
Y = data['Y']

#use pre-training to determine initial update order 
init_params = {}
init_params['noise'] = noise
init_params['iLatent'] = SP.where(data['terms']=='hidden')[0]

#get initial ordering via pre-training
Ilabel = utils.preTrain(Y, data['terms'], data['pi'],init_params)

#re-order terms
print "Initial order", data['terms'][Ilabel]
terms = data['terms'][Ilabel]
pi = data['pi'][:,Ilabel]


#initialise model
init={'init_data':fscLVM.CGauss(Y),'Pi':pi,'init_factors':init_params}
priors = {'Eps': {'priors':[1E-3,1E-3]}}
FA = fscLVM.CSparseFA(components=pi.shape[1], priors=priors,verbose=True)   
FA.init(**init) 

#iterate
FA.iterate()


We then plot results. First, we show the relevance of the terms; then we plot the 2 most important factors, in this case G2M Checkpoint and P53 Pathway. 

In [ ]:
#scatter plot of two most important factors
utils.plotTerms(FA, terms=terms)

dataFile = h5py.File(os.path.join(data_dir, dFile), 'r')
utils.plotFactors(0,1,FA,lab = dataFile['Known'][:][0,:]+2*dataFile['Known'][:][1,:], 
                  terms=terms, isCont=False)

Next, we can look how a Bayesian GPLVM looks like when we regress out confounding facotrs.

In [ ]:
import GPy
#Get model residuals
Ycorr = utils.regressOut(Y, idx=[0,2,6],FA=FA)

## Model optimization
Ystd = Ycorr-Ycorr.mean(0)
#Ystd/=Ystd.std(0)
input_dim = 2 # How many latent dimensions to use
kern = GPy.kern.RBF(input_dim,ARD=True) # ARD kernel
m = GPy.models.BayesianGPLVM(Ystd, input_dim=input_dim, kernel=kern, num_inducing=40)
m.optimize('scg', messages=1, max_iters=2000)

In [ ]:
import pylab as PL
PL.scatter(m.X.mean[:,0], m.X.mean[:,1], 40, dataFile['Known'][3,:])
PL.xlabel('Component 1')
PL.ylabel('Component 2')
PL.colorbar()

In [ ]:
#print the terms in order
print terms 

Q1: repeat this using a differnet annotation file by replacing MSigDB with REACTOME

Q2: Repeat the analysis on a different dataset of staged mESCs - which factors are most relevant now? What happens when you regress out cell cycle?
Hint: use the hdf5 file Buettneretal.hdf5

Q2.2: Colour the most imporant factors by cell cycle.
Hint: use dataFile['Known'][:][0,:]+2*dataFile['Known'][:][1,:] as color argument and plot the 2 most important components in the scatter plot 